In [32]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
#import random
from __future__ import print_function
from nltk.stem import *
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

TFlearn: Tt is a modular and transparent deep learning library built on top of Tensorflow. It was designed to provide a higher-level API to TensorFlow in order to facilitate and speed-up experimentations, while remaining fully transparent and compatible with it.
Natural Language Toolkit: NLTK is a open source Python library and Stemmers remove morphological affixes from words, leaving only the word stem. which is very importent when we are building a bot, because people may intract with in the way they way they normaly speak.
Pickle: It is used for serializing and de-serializing a Python object structure. Any object in python can be pickled so that it can be saved on disk. What pickle does is that it “serialises” the object first before writing it to file. Pickling is a way to convert a python object (list, dict, etc.) into a character stream.

In [33]:
# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

Json : it is java script notation we use json for giving data to the chat bot (I used json only because i dont know anyother way and it is a effetiant way and can we can feed much bigger data and easy to feed the data)

In [34]:
import nltk
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

27 documents
8 classes ['CNN', 'Machine learning', 'RNN', 'goodbye', 'greeting', 'payments', 'subjects', 'thanks']
60 unique stemmed words ["'d", "'s", 'a', 'about', 'acceiv', 'act', 'algorithem', 'an', 'anyon', 'ar', 'bye', 'card', 'cash', 'cnn', 'credit', 'day', 'diffr', 'do', 'doe', 'favourit', 'funct', 'good', 'goodby', 'hello', 'help', 'hi', 'how', 'i', 'in', 'intrest', 'is', 'kind', 'know', 'lat', 'lik', 'lm', 'mastercard', 'mor', 'most', 'of', 'on', 'rnn', 'see', 'suat', 'subject', 'tak', 'thank', 'that', 'the', 'ther', 'to', 'top', 'typr', 'us', 'what', 'when', 'which', 'work', 'yo', 'you']


understand and making a sence out of our data given though the Json file (intents.json)

In [27]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [35]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.10113 | time: 0.019s
| Adam | epoch: 1000 | loss: 0.10113 - acc: 0.9931 -- iter: 24/27
Training Step: 4000  | total loss: 0.09912 | time: 0.026s
| Adam | epoch: 1000 | loss: 0.09912 - acc: 0.9938 -- iter: 27/27
--
INFO:tensorflow:E:\bot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


Above is our neural deep neural network model which is just a normal DNN 
This is only the model it is trained and gives the prediction of the bots answer but doesnt gives out the answer. for bot to give out the response we must upgrade the code(I will update the code very soon, sorry for the delay)

In [36]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [37]:
p = bow("What subjects are you intrested in?", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1]
['CNN', 'Machine learning', 'RNN', 'goodbye', 'greeting', 'payments', 'subjects', 'thanks']


In [38]:
print(model.predict([p]))

[[5.7008136e-03 6.8853595e-03 4.6252115e-03 1.2328040e-02 4.1967167e-05
  6.7694644e-03 9.5863026e-01 5.0190105e-03]]
